<a href="https://colab.research.google.com/github/Gregrs400/cmpsc472Project2/blob/set-fire/cmpsc472Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
%%writefile cmpsc472Project2.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/wait.h>
#include <sys/time.h>
#include <sys/resource.h>


/*
  TODO:
  - add functionality to have this function wait an arbitrary amount of time before randomly setting another position equal to 1.
*/
void setFire(int array[20][20], int max, int min) {
  // Randomly select a row
  int row = rand() % (max - min + 1) + min;
  // Randomly select a column
  int col = rand() % (max - min + 1) + min;
  // Set the value at the selected row and column to 1
  array[row][col] = 1;
  printf("array value at position row,col: %d", array[row][col]);
}


int main() {
  // Define number of rows
  int rows = 20;
  // Define number of columns
  int columns = 20;
  // Create array to represent the landscape
  int landscape[rows][columns];
  // Fill array with 0's
  for (int i = 0; i < rows; i++) {
    for (int j = 0; j < columns; j++) {
      landscape[i][j] = 0;
    }
  }
  printf("array value at position 12,15: %d", landscape[12][15]);
  setFire(landscape, rows, columns);
  return 0;
}


Overwriting cmpsc472Project2.c


In [20]:
%%shell
gcc cmpsc472Project2.c -o cmpsc472Project2
./cmpsc472Project2

array value at position 12,15: 0array value at position row,col: 1